In [1]:
# Do the imports at the top

import random
from zenml import ExternalArtifact, pipeline 
from zenml.client import Client
from zenml.logger import get_logger
from uuid import UUID

import os
from typing import Optional, List

from zenml import pipeline

from pipelines import feature_engineering

from steps import (
    data_loader,
    data_preprocessor,
    data_splitter,
    model_evaluator,
    model_trainer,
    inference_predict,
    inference_preprocessor
)

logger = get_logger(__name__)

client = Client()

NumExpr defaulting to 8 threads.


# Start local with a simple training pipeline

First, lets run our training pipeline locally

<img src="_assets/default_stack.png" alt="Drawing" style="width: 800px;"/>

In [2]:
# To start with, we use the default stack
!zenml init
!zenml stack set default

# We also need to connect to a remote ZenML Instance
# !zenml connect --url ...

NumExpr defaulting to 8 threads.
Found existing ZenML repository at path 
'/home/apenner/PycharmProjects/zenml-projects/stack-showcase'.
⠋ Initializing ZenML repository at 
/home/apenner/PycharmProjects/zenml-projects/stack-showcase.
⠋ Initializing ZenML repository at 
/home/apenner/PycharmProjects/zenml-projects/stack-showcase.

NumExpr defaulting to 8 threads.
Active repository stack set to: 'default'.
⠙ Setting the repository active stack to 'default'...t'...


In [3]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from zenml import step
from zenml.logger import get_logger

logger = get_logger(__name__)


@step
def data_loader() -> pd.DataFrame:
    """Dataset reader step."""
    dataset = load_breast_cancer(as_frame=True)
    inference_size = int(len(dataset.target) * 0.05)
    dataset: pd.DataFrame = dataset.frame
    dataset.reset_index(drop=True, inplace=True)
    logger.info(f"Dataset with {len(dataset)} records loaded!")

    ### YOUR CODE ENDS HERE ###
    return dataset


In [ ]:
data_loader()

In [4]:
from zenml.config import DockerSettings

docker_settings = DockerSettings(
    requirements=[
        "pyarrow",
    ],
)

@pipeline(settings={"docker": docker_settings})
def training(
    train_dataset_id: Optional[UUID] = None,
    test_dataset_id: Optional[UUID] = None,
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
):
    """Model training pipeline."""
    # Execute Feature Engineering Pipeline
    dataset_trn, dataset_tst = feature_engineering()

    model = model_trainer(
        dataset_trn=dataset_trn,
    )

    model_evaluator(
        model=model,
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        min_train_accuracy=min_train_accuracy,
        min_test_accuracy=min_test_accuracy,
    )


In [ ]:
pipeline_args = {"enable_cache": False}
pipeline_args["config_path"] = os.path.join("configs", "training.yaml")
fe_t_configured = training.with_options(**pipeline_args)

In [ ]:
fe_t_configured()

# Let's outsource some compute to Sagemaker!

Let's farm some compute to AWS with a training job with a certain number of CPUs and Memory














<img src="_assets/local_sagmaker_so_stack.png" alt="Drawing" style="width: 800px;"/>

In [5]:
# This pip installs the requirements locally
!zenml integration install aws s3 -y

# This changes the active stack
!zenml stack set local-sagemaker-step-operator-stack

NumExpr defaulting to 8 threads.
⠧ Installing integrations...Requirement already satisfied: sagemaker==2.117.0 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (2.117.0)
⠏ Installing integrations...Collecting attrs<23,>=20.3.0
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
⠋ Installing integrations...Collecting argparse
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
⠙ Installing integrations...Requirement already satisfied: jmespath<2.0.0,>=0.7.1 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from boto3<=1.26.76) (1.0.1)
⠹ Installing integrations...Requirement already satisfied: cachetools<6.0,>=2.0.0 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from google-auth>=1.0.1->kubernetes) (5.3.1)
⠸ Installing integrations...Requirement already satisfied: pytz>=2020.1 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from pandas->sagemaker==2.117.0) (202

In [6]:
step_args = {}
step_args["step_operator"] = "sagemaker-eu"

# M5 Large is what we need for this big data!
step_args["settings"] = {"step_operator.sagemaker": {"estimator_args": {"instance_type" : "ml.m5.large"}}}

# Update the step. We can also do this in YAML
model_trainer = model_trainer.with_options(**step_args)
    
pipeline_args = {"enable_cache": False}
pipeline_args["config_path"] = os.path.join("configs", "training.yaml")
fe_t_configured = training.with_options(**pipeline_args)

In [ ]:
fe_t_configured()

# Let's run the entire pipeline on Airflow now


<img src="_assets/airflow_stack.png" alt="Drawing" style="width: 800px;"/>


In [7]:
!zenml stack set sagemaker-airflow-stack
!zenml integration install airflow -y
!pip install apache-airflow-providers-docker apache-airflow~=2.5.0
!zenml stack up

NumExpr defaulting to 8 threads.
Active repository stack set to: 'sagemaker-airflow-stack'.
⠴ Setting the repository active stack to 'sagemaker-airflow-stack'...k'...
NumExpr defaulting to 8 threads.
⠏ Installing integrations.....Collecting apache-airflow~=2.4.0
  Using cached apache_airflow-2.4.3-py3-none-any.whl (6.5 MB)
⠋ Installing integrations...Requirement already satisfied: cattrs>=22.1.0 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from apache-airflow~=2.4.0) (23.2.3)
⠙ Installing integrations...Requirement already satisfied: python-dateutil>=2.3 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from apache-airflow~=2.4.0) (2.8.2)
⠹ Installing integrations...Requirement already satisfied: exceptiongroup>=1.1.1 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from cattrs>=22.1.0->apache-airflow~=2.4.0) (1.1.3)
⠸ Installing integrations...  Using cached attrs-23.1.0-py3-none-any.whl (61 k

In [8]:
fe_t_configured()

Initiating a new run for the pipeline: training.
Reloading configuration file /home/apenner/PycharmProjects/zenml-projects/stack-showcase/.zen/config.yaml
Reusing registered version: (version: 8).
New model version 11 was created.
Building Docker image(s) for pipeline training.
Building Docker image 715803424590.dkr.ecr.eu-central-1.amazonaws.com/zenml:training-orchestrator.
- Including user-defined requirements: pyarrow
- Including integration requirements: apache-airflow~=2.4.0, aws-profile-manager, boto3<=1.26.76, kubernetes, s3fs>2022.3.0,<=2023.4.0, sagemaker==2.117.0, scikit-learn<1.3
Could not import Azure service connector: No module named 'azure.identity'.
Step 1/10 : FROM zenmldocker/zenml:0.50.0-py3.9
Step 2/10 : WORKDIR /app
Step 3/10 : COPY .zenml_user_requirements .
Step 4/10 : RUN pip install --default-timeout=60 --no-cache-dir  -r .zenml_user_requirements
Step 5/10 : COPY .zenml_integration_requirements .
Step 6/10 : RUN pip install --default-timeout=60 --no-cache-dir  

# Switch to full Sagemaker Stack

![Sagemaker local stack](_assets/sagemaker_stack.png)


In [ ]:
!zenml stack set sagemaker-stack

In [ ]:
fe_t_configured()